In [279]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mock-test-2-mse-2/sample_submission.csv
/kaggle/input/mock-test-2-mse-2/train.csv
/kaggle/input/mock-test-2-mse-2/test.csv


In [280]:

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,log_loss

In [281]:
df = pd.read_csv("/kaggle/input/mock-test-2-mse-2/train.csv")
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1481.0,Placebo,12963.0,F,N,Y,N,N,2.4,346.0,3.34,70.0,1212.0,122.45,118.0,117.0,12.2,4.0,C
1,1,2580.0,D-penicillamine,20819.0,F,N,N,N,N,0.6,NaN,4.52,51.0,645.0,74.40,NaN,181.0,10.4,3.0,C
2,2,837.0,D-penicillamine,12307.0,F,N,Y,Y,N,6.1,586.0,2.73,89.0,2045.0,196.85,90.0,228.0,10.4,3.0,D
3,3,3021.0,NaN,17532.0,F,NaN,NaN,NaN,N,0.6,NaN,3.34,NaN,NaN,NaN,NaN,388.0,10.4,3.0,C
4,4,1980.0,NaN,23011.0,F,NaN,NaN,NaN,N,0.8,NaN,3.65,NaN,NaN,NaN,NaN,190.0,11.1,4.0,C


In [282]:
df.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,15000.000000,15000.00000,15000.000000,15000.000000,6625.000000,15000.000000,8343.000000,8453.000000,8449.000000,6578.000000,14418.000000,14981.000000,15000.000000
mean,7499.500000,1957.65580,19340.767667,1.869133,327.408755,3.523682,74.437732,1671.871596,106.072495,110.992703,253.589402,10.624471,3.037200
std,4330.271354,1346.32124,3731.980408,2.716933,172.503702,0.368956,72.604081,1874.328641,55.331238,50.127109,92.802002,0.728577,0.877497
min,0.000000,28.00000,1223.000000,0.300000,44.000000,1.960000,2.000000,140.000000,26.350000,4.000000,2.000000,9.000000,1.000000
25%,3749.750000,1095.00000,16714.000000,0.700000,242.000000,3.290000,31.000000,724.000000,71.000000,80.000000,183.000000,10.100000,2.000000
50%,7499.500000,1770.00000,19698.000000,0.900000,280.000000,3.580000,52.000000,1080.000000,97.650000,99.000000,249.000000,10.600000,3.000000
75%,11249.250000,2609.00000,22388.000000,1.800000,364.000000,3.770000,88.000000,1692.000000,130.200000,133.000000,311.000000,11.000000,4.000000
max,14999.000000,25569.00000,127321.000000,28.000000,2580.000000,4.640000,674.000000,20392.000000,2656.700000,598.000000,824.000000,18.000000,4.000000


In [283]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8459 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8460 non-null   object 
 6   Hepatomegaly   8457 non-null   object 
 7   Spiders        8448 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6625 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8343 non-null   float64
 13  Alk_Phos       8453 non-null   float64
 14  SGOT           8449 non-null   float64
 15  Tryglicerides  6578 non-null   float64
 16  Platelets      14418 non-null  float64
 17  Prothrombin    14981 non-null  float64
 18  Stage 

In [284]:
df["Status_bin"] = df["Status"].map({"D": 0, "C": 1, "CL": 2})

In [285]:
X = df.drop(["Status", "Status_bin", "id"], axis=1)

y = df["Status_bin"]

In [286]:
cat_cols = X.select_dtypes(include=["object"]).columns

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # ensure string type
    label_encoders[col] = le

In [287]:
X = X.fillna(X.median(numeric_only=True))

In [288]:
def cap_outliers(df, cols, lower=1, upper=99):
    for col in cols:
        low_val = df[col].quantile(lower/100)
        high_val = df[col].quantile(upper/100)
        df[col] = df[col].clip(low_val, high_val)
    return df

num_cols = X.select_dtypes(include=['float64','int64']).columns
X = cap_outliers(X, num_cols)

In [289]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42, stratify=y)

In [290]:
# model = RandomForestClassifier(n_estimators=200,random_state=42)
# model.fit(X_train, y_train)



In [291]:
# from sklearn.calibration import CalibratedClassifierCV

# clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
# model = CalibratedClassifierCV(base_estimator=clf, method='isotonic', cv=5)

# model.fit(X_train, y_train)


In [292]:
# from sklearn.calibration import CalibratedClassifierCV

# base = lgb.LGBMClassifier(
#     objective='multiclass',
#     metric='multi_logloss',
#     num_class=num_classes,
#     n_estimators=800,
#     learning_rate=0.05,
#     num_leaves=31
# )

# model = CalibratedClassifierCV(base, method='isotonic', cv=5)

# model.fit(X_train, y_train)   # <— this trains AND calibrates


In [293]:
import lightgbm as lgb
from sklearn.calibration import CalibratedClassifierCV

num_classes = len(np.unique(y_train))

# STEP 1: Define base model (do NOT fit it)
clf = lgb.LGBMClassifier(
    objective='multiclass',
    metric='multi_logloss',
    num_class=num_classes,
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)


model = CalibratedClassifierCV(base_estimator=clf, method='isotonic', cv=5)


model.fit(X_train, y_train)




/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1816
[LightGBM] [Info] Number of data points in the train set: 11760, number of used features: 18
[LightGBM] [Info] Start training from score -1.200577
[LightGBM] [Info] Start training from score -0.394177
[LightGBM] [Info] Start training from score -3.699136
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001614 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1827
[LightGBM] [Info] Number of data points in the train set: 11760, number of used features: 18
[LightGBM] [Info] Start training from score -1.200577
[LightGBM] [Info] Start training from score -0.394177
[LightGBM] [Info] Start training from score -3.699136
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001627 sec

CalibratedClassifierCV(base_estimator=LGBMClassifier(colsample_bytree=0.8,
                                                     learning_rate=0.01,
                                                     metric='multi_logloss',
                                                     n_estimators=1000,
                                                     n_jobs=-1, num_class=3,
                                                     objective='multiclass',
                                                     random_state=42,
                                                     reg_alpha=0.1,
                                                     reg_lambda=1.0,
                                                     subsample=0.8),
                       cv=5, method='isotonic')

In [294]:
y_pred =model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
acc

0.8533333333333334

In [295]:
y_pred_prob = model.predict_proba(X_test)
log_l = log_loss(y_test,y_pred_prob,labels = model.classes_)
log_l


0.3440692259817818

In [296]:
df_test = pd.read_csv("/kaggle/input/mock-test-2-mse-2/test.csv")
df_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,15000,1614.0,Placebo,21281.0,F,N,N,N,N,0.6,257.0,3.50,44.0,1345.0,93.00,68.0,NaN,9.7,4.0
1,15001,2573.0,D-penicillamine,20834.0,F,N,N,N,N,0.6,217.0,4.19,13.0,678.0,45.00,75.0,356.0,9.6,3.0
2,15002,1367.0,NaN,23011.0,F,NaN,NaN,NaN,N,0.8,NaN,3.03,NaN,NaN,NaN,NaN,312.0,10.6,2.0
3,15003,1080.0,Placebo,15895.0,M,N,N,N,N,2.3,201.0,3.55,44.0,1623.0,237.15,56.0,224.0,13.6,2.0
4,15004,3853.0,Placebo,14872.0,F,N,N,N,N,0.7,259.0,3.77,44.0,685.0,97.65,93.0,249.0,11.0,1.0


In [297]:
x_test=df_test.drop(["id"], axis=1)

test_ids = df_test["id"]         
x_test = df_test.drop(["id"], axis=1)


for col, le in label_encoders.items():
    if col in x_test.columns:
       
        class_to_int = {cls: idx for idx, cls in enumerate(le.classes_)}
        x_test[col] = x_test[col].astype(str).map(class_to_int)


x_test = x_test.fillna(x_test.median(numeric_only=True))


In [298]:
y_pred1 = model.predict(x_test)

y_pred1

array([1, 1, 1, ..., 1, 0, 1])

In [299]:
y_pred_prob1 = model.predict_proba(x_test)

In [300]:


class_labels = model.classes_

submission_cols = [f"Status_{label}" for label in class_labels]

submission = pd.DataFrame(y_pred_prob1, columns=submission_cols)

submission.insert(0, "id", test_ids)
submission = submission.rename(columns={
    "Status_0": "Status_D",
    "Status_1": "Status_C",
    "Status_2": "Status_CL"
})

# Reorder columns to: id, Status_C, Status_CL, Status_D
submission = submission[["id", "Status_C", "Status_CL", "Status_D"]]



submission.to_csv("professor.csv", index=False)

print("Submission file saved as submission.csv")
print(submission.head())

Submission file saved as submission.csv
      id  Status_C  Status_CL  Status_D
0  15000  0.983320   0.001041  0.015640
1  15001  0.974819   0.004033  0.021147
2  15002  0.920133   0.010789  0.069078
3  15003  0.125597   0.014851  0.859552
4  15004  0.987293   0.001062  0.011645
